In [1]:
# General libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Models libraries
from sklearn import svm
# Models tuning libraries
from sklearn.model_selection import GridSearchCV

## Preliminary work on the dataset

In [2]:
yahoo_df = pd.read_csv("../data/OIL.csv")
yahoo_df = yahoo_df.set_index("Date")

X = yahoo_df[['ROC_1', 'ROC_4', 'ROC_5', 'ROC_30', 'Return', '2 Day ROI', '3 Day ROI',
       '4 Day ROI', '30 Day ROI', 'RSI_1', 'RSI_2', 'RSI_3', 'RSI_4',
       'MACDs_12_26_9', 'SRSI_14', 'SRSI_30', 'CCI', 'Previous_differenced',
       'principalDf']]

y = yahoo_df["Up down"]

# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)

# Normalize

In [3]:
def shift_norm(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [4]:
X_train_80 = shift_norm(X_train_80)
X_test = shift_norm(X_test)

In [5]:
X_train = shift_norm(X_train)
X_valid = shift_norm(X_valid)

# Support Vector Machine (SVC)

In [6]:
SVC = svm.SVC(random_state = 42, kernel='linear')

# Fit the model
SVC.fit(X_train, y_train)

# Predict
y_pred_train = SVC.predict(X_train)
y_pred_valid = SVC.predict(X_valid)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

Train Accuracy: 0.53785 - Validation Accuracy: 0.53912


In [7]:
y_pred_test = SVC.predict(X_test)
# Finally evaluate on test
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

# Store the Test Accuracy 
SVC_accuracy_test = test_acc

Test Accuracy: 0.52050


### Tuning parametres with GridSearchCV(VERY SLOW!!)

We make parameters tuning using the GridSearchCV instance. This one allows us to fit on a dataset all the possible combinations of parameter values. Moreover it evaluates and retaines the best combination of parameters.

# Applying Gridsearchcv for parameters tuning
# 'poly' --> molto lento
# Setting C parameters
parameters = [{'C': [0.1, 1.0, 10, 50, 100 ],
              'gamma': [1,0.1,0.001],
                'kernel': ['linear','rbf', 'sigmoid',"poly"]
              }] 
best_SVC = GridSearchCV(estimator = svm.SVC(),refit=True,verbose=2,
                             param_grid = parameters, scoring = 'accuracy')

# Fit the model
best_SVC.fit(X_train, y_train)

print(best_SVC.best_estimator_)

In [8]:
SVC = svm.SVC(C=10, gamma=1, kernel='linear',probability=True)

# Fit the model
SVC.fit(X_train_80, y_train_80)

# Predict
y_pred_train = SVC.predict(X_train)
y_pred_valid = SVC.predict(X_valid)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

Train Accuracy: 0.55025 - Validation Accuracy: 0.55057


In [9]:
y_pred_test = SVC.predict(X_test)
y_prob = SVC.predict_proba(X_test)[:,1]

# Finally evaluate on test
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

# Store the Test Accuracy 
SVC_accuracy_test = test_acc

Test Accuracy: 0.52240


# Predict on all data 

In [10]:
df_prediction = pd.read_csv("../data/model_accuracy_oil.csv", index_col = False)
df_prediction =  df_prediction.drop(columns=["Unnamed: 0"])

df_prediction

,ANN_pred,ANN_accuracy,ANN_prob,y,kNN_pred,kNN_accuracy,kNN_prob,RF_pred,RF_accuracy,RF_prob,XGB_pred,XGB_accuracy,XGB_prob,LGBM_pred,LGBM_accuracy,LGBM_prob
0,0,0.469018,0.404047,0,1,0.488084,1.0,0,0.514776,0.440374,0,0.521449,0.499819,1,0.533842,0.513085
1,0,0.469018,0.342436,1,0,0.488084,0.0,1,0.514776,0.558104,1,0.521449,0.511250,1,0.533842,0.513026
2,0,0.469018,0.339922,1,0,0.488084,0.0,1,0.514776,0.565385,0,0.521449,0.494657,1,0.533842,0.513026
3,0,0.469018,0.404326,0,1,0.488084,1.0,1,0.514776,0.522251,1,0.521449,0.500281,1,0.533842,0.504916
4,0,0.469018,0.361674,0,1,0.488084,1.0,1,0.514776,0.583792,1,0.521449,0.529373,1,0.533842,0.515756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0,0.469018,0.448136,0,0,0.488084,0.5,0,0.514776,0.470678,1,0.521449,0.549180,1,0.533842,0.506586
1045,0,0.469018,0.448477,1,0,0.488084,0.5,0,0.514776,0.485184,1,0.521449,0.531798,1,0.533842,0.505718
1046,0,0.469018,0.438599,0,0,0.488084,0.5,0,0.514776,0.466790,1,0.521449,0.512161,1,0.533842,0.506459
1047,0,0.469018,0.417918,1,0,0.488084,0.0,1,0.514776,0.589712,1,0.521449,0.542420,1,0.533842,0.508596


In [11]:
df_prediction["SVC_pred"] = y_pred_test
df_prediction["SVC_accuracy"] = [SVC_accuracy_test for x in range(len(y_pred_test))]
df_prediction["SVC_prob"] = y_prob


In [12]:
df_prediction

,ANN_pred,ANN_accuracy,ANN_prob,y,kNN_pred,kNN_accuracy,kNN_prob,RF_pred,RF_accuracy,RF_prob,XGB_pred,XGB_accuracy,XGB_prob,LGBM_pred,LGBM_accuracy,LGBM_prob,SVC_pred,SVC_accuracy,SVC_prob
0,0,0.469018,0.404047,0,1,0.488084,1.0,0,0.514776,0.440374,0,0.521449,0.499819,1,0.533842,0.513085,1,0.522402,0.529821
1,0,0.469018,0.342436,1,0,0.488084,0.0,1,0.514776,0.558104,1,0.521449,0.511250,1,0.533842,0.513026,1,0.522402,0.509924
2,0,0.469018,0.339922,1,0,0.488084,0.0,1,0.514776,0.565385,0,0.521449,0.494657,1,0.533842,0.513026,0,0.522402,0.491893
3,0,0.469018,0.404326,0,1,0.488084,1.0,1,0.514776,0.522251,1,0.521449,0.500281,1,0.533842,0.504916,0,0.522402,0.500000
4,0,0.469018,0.361674,0,1,0.488084,1.0,1,0.514776,0.583792,1,0.521449,0.529373,1,0.533842,0.515756,1,0.522402,0.515805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0,0.469018,0.448136,0,0,0.488084,0.5,0,0.514776,0.470678,1,0.521449,0.549180,1,0.533842,0.506586,1,0.522402,0.532198
1045,0,0.469018,0.448477,1,0,0.488084,0.5,0,0.514776,0.485184,1,0.521449,0.531798,1,0.533842,0.505718,1,0.522402,0.546324
1046,0,0.469018,0.438599,0,0,0.488084,0.5,0,0.514776,0.466790,1,0.521449,0.512161,1,0.533842,0.506459,1,0.522402,0.518366
1047,0,0.469018,0.417918,1,0,0.488084,0.0,1,0.514776,0.589712,1,0.521449,0.542420,1,0.533842,0.508596,1,0.522402,0.554229


In [13]:
df_prediction.to_csv("../data/model_accuracy_oil.csv")

In [14]:
# Plot the confusion matrix
#conf_stat = confusion_matrix(y_test, y_pred_test)
#ut.plot_confusion_matrix(conf_stat, Linearmodel_nt.classes_)